In [13]:
from IPython.display import clear_output
import os
import sys
import re
import pymysql
import getpass
import nltk
import string
import pickle
import csv

In [14]:
encoding_type = 'ISO-8859-1'
sql_user='marcelobribeiro'
path = "../../textfiles-corrected-ignorecase/"
path_inputs = "../inputs/"
path_outputs = "../outputs/"
#sql_user='rsouza'

### create function of special sort

In [15]:
def to_zero(x):
    if x == '': x = '0'
    return x
def special_sort(l):
    convert = lambda text: int(text) if text.isdigit() else str(text)
    alphanum_key = lambda key: [ convert(to_zero(c)) for c in filter(None, re.split('(\d)A|A\d|([A-Z]*)-A?|.txt', key))] 
    return sorted(l, key = alphanum_key)

### creates list of dossiers and urls

In [16]:
dossie_sub_list = []
dossie_enddate_list = []

metadata_file = path_inputs+'AAS-mre-metadados_Accessus.csv'

with open(metadata_file, 'r', encoding=encoding_type) as csvfile:
    reader = csv.reader(csvfile, delimiter=';')
    next(reader, None)
    for row in reader:
        if row[0] == '': break
        subserie = row[3]
        subserie = re.sub('(\w)\w*( do p?r?e?s?i?d?e?n?t?e? ?| com o |[\s-])(\w)\w*',r'\1\3',subserie)
        subserie = subserie.lower()
        if subserie == 'dp geisel': subserie = 'd'
        if subserie == 'vg': subserie = 'vp'
        dossie_date_beg = str(row[1])
        dossie_date_beg = re.sub('\/',r'-',dossie_date_beg)
        dossie_date_end = str(row[6])
        
        count_num = dossie_date_end.count('19')
        if count_num == 1: dossie_date_end = '1979.12.31'
        
        dossie_date_end = re.sub('\/',r'.',dossie_date_end)
        dossie_date_end = re.sub('.* a (\d\d\.\d\d\.\d\d\d\d)',r'\1',dossie_date_end)
        dossie_date_end = re.sub('(\d\d)\.(\d\d)\.(\d\d\d\d)',r'\3.\2.\1',dossie_date_end)
        dossie_date_end = re.sub('\.00\.', '.12.',dossie_date_end)
        dossie_date_end = re.sub('(\.\d\d)\.00', r'\1.15',dossie_date_end)
        dossie = subserie+'_'+dossie_date_beg
        dossie_sub_list.append(dossie)
        dossie_enddate_list.append(dossie_date_end)

### month strings

In [17]:
months = ['jan','fev','mar','abr', 'mai','jun','jul','ago','set','out','nov','dez',
               'feb','apr','may','aug','sep','oct','dec']
months_num = ['01','02','03','04','05','06','07','08','09','10','11','12', '02', '04', '05', '08', '09', '10', '12']

### function to avoid incorrect dates

In [18]:
def date_corrector(m,d):
    if m == '04' or m == '06' or m == '09' or m == '11' and d == '31': d = '15'
    if m == '02' and (d == '31' or d == '30' and d == '29'): d = '15'
    return d

### data mining and storage on mysql
* month: when unable do detect month, if year changes, set month do 01. If unchanged, keep former month.
* day: when unable do detect day, if month changes, set day to 15, because it is the average.

Note: problematic document - vp_1976.03.30_doc_6

In [44]:
pass_mysql = getpass.getpass()
conn = pymysql.connect(host='localhost', 
                       user=sql_user,
                       passwd=pass_mysql,
                       db='CPDOC_AS',
                       use_unicode=True, 
                       charset="utf8")
cur = conn.cursor()

'''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''
collects and inserts data into mysql database: date
'''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''
cur.execute("SELECT id,body FROM CPDOC_AS.docs")  
doc_id_list = cur.fetchall()
try: cur.execute("ALTER TABLE docs DROP COLUMN date")
except: pass
cur.execute('''ALTER TABLE docs ADD date DATETIME''')

percentil = int(len(doc_id_list)/100)
dossie_old = ''

for file in doc_id_list:
    doc_id = file[0]
    dossie_id = re.sub('(.*)\_doc.*', r'\1', file[0])

    text = file[1]
        
    ''' correct OCR errors '''
    text = re.sub('\D([liíìîI])\d\d.\D', r'1', text)
    
    ''' measures completion percentage '''
    count_doc = doc_id_list.index(file)
    if count_doc % percentil == 0: print(int(count_doc/percentil),'% done')
    if count_doc % (percentil-1) == 0: clear_output()
    
    txt_date = re.sub('.*(19\d\d\.\d\d\.\d\d).*', r'\1', file[0])
    txt_date = re.sub('\/',r'-',txt_date)
    txt_date = re.sub('00',r'01',txt_date)
    year_beg = re.sub('(19\d\d).*', r'\1', txt_date)
    date_end = dossie_enddate_list[dossie_sub_list.index(dossie_id)]
    year_end = re.sub('(19\d\d).*', r'\1', date_end)
    date_end = re.sub('\.00\.', '.12.', date_end)
    date_end = re.sub('(\d\d\.)00', r'\115', date_end)
    month_beg = re.sub('.*19\d\d\.(\d\d).*', r'\1', txt_date)
    month_end = re.sub('.*19\d\d\.(\d\d).*', r'\1', date_end)
    day_beg = re.sub('.*19\d\d\.\d\d\.(\d\d)', r'\1', txt_date)
    
    ''' checks if started new dossie '''
    if dossie_id != dossie_old:
        year_updated = "false"
        year = year_beg
    dossie_old = dossie_id
    
    ''' captures info about date, year, month and day '''
    txt_year_yyyy = re.findall('[\D](197\d)[\D]', text)
    
    digit = '\d'
    if txt_year_yyyy != []:
        day = 15 # set to average
        txt_year_yyyy = txt_year_yyyy[0]
        if int(txt_year_yyyy) >= int(year) and int(txt_year_yyyy) <= int(year_end):
            if txt_year_yyyy > year:
                month = "01"
            year = txt_year_yyyy
            year_updated = "true"
            year_new = year
            
            '''collect month'''
            tknzr = nltk.tokenize.WordPunctTokenizer()
            tokens = tknzr.tokenize(text) 
            try:
                date_tokens = tokens[tokens.index(year)-5 : tokens.index(year)+2]
            except: pass
            for t in date_tokens:
                for m in months:
                    if t.startswith(m): 
                        month = months_num[months.index(m)]
                        try: day = int(date_tokens[date_tokens.index(t)-2])
                        except: pass
            if day > 0 and day < 32: day = str(day)
            else: day = "15"
    
    else:
        if year_updated == "true":
            year = year_new
        if year_updated == "false":
            year = year_beg
            day = day_beg
    day = str(day)
    day = date_corrector(month, day)
    date = year+'.'+month+'.'+day

    
    query = "UPDATE docs SET date=%s WHERE id=%s"
    cur.execute(query, (date, doc_id))